Investigate the influence of different training parameters such as batch size and learning rate.
Perform a  random search to find the optimal training
parameters.

RNN model

In [23]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import PreProcessor as pp
import gc
from keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Dense, Lambda, Softmax, Activation, concatenate
from keras import optimizers
import tensorflow as tf
from keras_tuner import RandomSearch
import utils as ut



In [4]:
# Définir le modèle
#Loss plot w.r.t. the learning rate
dataset = pp.load_dataset(r"C:\Users\anhly\Desktop\DD2424-MusicGen1/adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity = pp.dicts_from_songs(ClassicSongs)
time_range = pp.ranges_from_songs(ClassicSongs)

n_Channels = len(channel_to_ind)
n_Notes = len(note_to_ind)
n_Velocities = len(velocity_to_ind)


dim = 250
#RNN_model = ut.create_model("RNN",dim,n_Channels,n_Notes,n_Velocities,time_range)

In [19]:
print(velocity_to_ind)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121,

In [24]:
seq_length=30
X_Channels, X_Notes, X_Velocities, X_Times, y_Channels, y_Notes, y_Velocities, y_Times, Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times, Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times, Test_X_Channels, Test_X_Notes, Test_X_Velocities, Test_X_Times, Test_y_Channels, Test_y_Notes, Test_y_Velocities, Test_y_Times = ut.prep_data(seq_length,ClassicSongs,n_Channels,n_Notes,channel_to_ind,note_to_ind,velocity_to_ind,val_split=0.1,test_split=0.1)

TypeError: prep_data() missing 1 required positional argument: 'velocity_to_ind'

In [4]:
losses = {
    'Channels': 'categorical_crossentropy',
    'Notes': 'categorical_crossentropy',
    'Velocities': 'categorical_crossentropy',
    'Times': 'mean_squared_error'
}


loss_weights = {
    'Channels': 0.25,
    'Notes': 5.0,
    'Velocities': 1.0,
    'Times': 1e-6
}

metrics = {
    'Channels': 'accuracy',
    'Notes': 'accuracy',
    'Velocities': 'accuracy',
    'Times': ut.tol_accuracy
}

In [13]:
print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])


Number of channels   : 12 
Number of notes      : 105 
Number of velocities : 128 
Time range           : 0.0 190.285


In [9]:
print(n_Channels)
print(time_range)
print(n_Notes)
print(n_Velocities)

12
[0.0, 190.285]
105
128


In [11]:
def build_model(hp):
    model = ut.create_model('RNN', dims=250, n_Channels=n_Channels, n_Notes=n_Notes, n_Velocities=2, time_range=time_range)
    
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss=losses,loss_weights=loss_weights,metrics=metrics)
    model.summary()
    return model

In [12]:
# Configurer la recherche aléatoire
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='rnn_hyperparameter_tuning'
)

Reloading Tuner from my_dir\rnn_hyperparameter_tuning\tuner0.json


In [13]:
# Définir le fit avec hyperparamètres
def fit_with_hp(hp):
    batch_size = hp.Int('batch_size', min_value=32, max_value=256, step=32)
    return model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.2)


In [14]:
# Charger les données
(x_train, y_train) = ([X_Channels, X_Notes, X_Velocities, X_Times], [y_Channels, y_Notes, y_Velocities, y_Times])


NameError: name 'X_Channels' is not defined

In [10]:
print(len(x_train))

4


In [11]:
print(X_Channels.shape)
print(X_Notes.shape)
print(X_Velocities.shape)
print(X_Times.shape)
print(Val_X_Times.shape)
print(Test_X_Times.shape)

(56724, 30, 12)
(56724, 30, 105)
(56724, 30, 1)
(56724, 30, 1)
(7090, 30, 1)
(7090, 30, 1)


In [12]:
# Lancer la recherche
tuner.search(x_train, y_train, epochs=20, batch_size=302)

# Récupérer les meilleurs hyperparamètres et entraîner le modèle final
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2)


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
4.1125e-05        |0.00098013        |learning_rate



Model: "RNN_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 12)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 105) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None, 128) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None, 246) │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
│                     │                   │            │ input_layer_2[0]… │
│                     │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn          │ (None, None, 250) │    124,250 │ concatenate[0][0] │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 12)  │      3,012 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 105) │     26,355 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None, 128) │     32,128 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None, 1)   │        251 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Channels (Softmax)  │ (None, None, 12)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Notes (Softmax)     │ (None, None, 105) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Velocities          │ (None, None, 128) │          0 │ dense_2[0][0]     │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Times (Lambda)      │ (None, None, 1)   │          0 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 185,996 (726.55 KB)

 Trainable params: 185,996 (726.55 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


Traceback (most recent call last):
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\hyperm

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Users\anhly\AppData\Local\Pr